<a href="https://colab.research.google.com/github/adventHymnals/resources/blob/master/Update_VegM_Website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Website Tools

Run this noteboook to update(push changes) to website once you have edited [the website config sheet](https://docs.google.com/spreadsheets/d/1-otVkXNn6i4tWGki9grXzc4DjG6TdQ_XWTBbJttlESU/edit#gid=512290428).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%bash
dirPath=""
if [ -d "/content/drive/MyDrive/Publishing/Website" ]
then
    dirPath="/content/drive/MyDrive/Publishing/Website"
else
    if [ -d "/content/drive/My Drive/Publishing/Website" ]
    then
        dirPath="/content/drive/My Drive/Publishing/Website"
    else
        exit
    fi
fi

#### ssh key
mkdir -p ~/.ssh
cp "$dirPath/private/id_rsa" ~/.ssh/

#### download google sheet
docId="1-otVkXNn6i4tWGki9grXzc4DjG6TdQ_XWTBbJttlESU"  ## Hardcode this (for now)
sheetId="1050384173"                                  ## Hardcode this (for now)
rm -f logs1
wget --output-file="logs1" "https://docs.google.com/spreadsheets/d/$docId/export?format=csv&gid=$sheetId" -O "sheet.csv"
rm logs1

#### process sheet vars
# prelink=$(grep -rh "Github," sheet.csv | sed 's/Github,//')
#prelink=$(cat sheet.csv |grep -o 'Github,[^ ]*' | cut -d',' -f2 | tr -d '\r')
#prelink=$(cat sheet.csv | awk -F',' '/Github/ {print $2}') | tr -d '\r'
prelink=$(cat sheet.csv |grep -o 'Github,[^ ]*' | cut -d',' -f2 | tr -d '\r')
link="git@github.com:$prelink.git"

GitCommitUser=$(grep -rh "GitCommitUser," sheet.csv | sed 's/GitCommitUser,//' | tr -d '\r')
GitCommitEmail=$(grep -rh "GitCommitEmail," sheet.csv | sed 's/GitCommitEmail,//'| tr -d '\r')
rm sheet.csv


#### clone repo
rm -rf websiteRepo
mkdir -p websiteRepo

ssh-keyscan GitHub.com >> /root/.ssh/known_hosts
chmod 644 /root/.ssh/known_hosts
chmod 600 /root/.ssh/id_rsa
git clone "$link" websiteRepo


#### Download sheets
cd websiteRepo
mkdir -p "./configs/sheets"
rm -f logs
while IFS=, read -r docId sheetId; do
    wget --output-file="logs" "https://docs.google.com/spreadsheets/d/$docId/export?format=csv&gid=$sheetId" -O "./configs/sheets/$docId$sheetId.csv"
done < "./configs/datasheets.txt"
rm -f logs

#### push repo
git config --global user.email "$GitCommitEmail" 
git config --global user.name "$GitCommitUser"
git add . && git commit -m "$(date)" && git push origin master

